In [ ]:
from datetime import date, datetime
from typing import Optional, Union
import sqlite3
from API.booking import BookingAPI
from API.geoCoding import GeoCodingAPI
from API.getKey import OAuthClient
from API.getQuotes import QuotesAPI
from API.is_Airport import IsAirport
from langchain_core.tools import tool
# @tool 
# def action_ask_confirm_info():
#     pass
@tool 
def validate_pick_up_location(location: str):
    geoCodingAPI = GeoCodingAPI()
    geoCoding_pickup = geoCodingAPI.get_geocoding(location)
    try:
        if geoCoding_pickup["status"] == "OK" :
            return {"pick_up_result": geoCoding_pickup}
        else:
            return {"pick_up_location": None}
    except Exception as e:
        print(e)
        return {"pick_up_location": None}
    

In [1]:
class BookingForm :
    def __init__(self):
        self.required_slots = ["name","number-phone","pickup_location", "destination_location", "pickup_time"]
        self.slots = {slot: None for slot in self.required_slots}

    def get_next_slot(self):
        for slot, value in self.slots.items():
            if value is None: 
                return slot
        return None 

    def handle_user_input(self, slot, user_input):
        self.slots[slot] = user_input

    def is_complete(self):
        return all(value is not None for value in self.slots.values())

    def run(self):
        print("Welcome! I need some information to book your ride.")

        while not self.is_complete():
            next_slot = self.get_next_slot()
            if next_slot:
                # Hỏi người dùng về slot tiếp theo
                user_input = input(f"Please provide your {next_slot.replace('_', ' ')}: ")
                self.handle_user_input(next_slot, user_input)

        print("Thank you! Here is the information you provided:")
        for slot, value in self.slots.items():
            print(f"{slot.replace('_', ' ').capitalize()}: {value}")


In [2]:
from typing import Annotated, Literal, Optional
from typing_extensions import TypedDict
from langgraph.graph.message import AnyMessage, add_messages

def update_dialog_stack(left: list[str], right: Optional[str]) -> list[str]:
    """Push or pop the state."""
    if right is None:
        return left
    if right == "pop":
        return left[:-1]
    return left + [right]

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    user_info: str
    dialog_state: Annotated[
        list[
            Literal[
                "assistant",
                "book_car",
            ]
        ],
        update_dialog_stack,
    ]

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig

from pydantic import BaseModel, Field

In [3]:
class ToBookCarRental(BaseModel):
    """Transfers work to a specialized assistant to handle car rental bookings."""

    location: str = Field(
        description="The location where the user wants to rent a car."
    )
    start_date: str = Field(description="The start date of the car rental.")
    end_date: str = Field(description="The end date of the car rental.")
    request: str = Field(
        description="Any additional information or requests from the user regarding the car rental."
    )

    class Config:
        json_schema_extra = {
            "example": {
                "location": "Basel",
                "start_date": "2023-07-01",
                "end_date": "2023-07-05",
                "request": "I need a compact car with automatic transmission.",
            }
        }

NameError: name 'BaseModel' is not defined